In [10]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
import numpy as np
import collections
%matplotlib inline

def live_plot(data_dict, figsize=(7,5), title=''):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    for label,data in data_dict.items():
        plt.plot(data, label=label)
    plt.title(title)
    plt.grid(True)
    plt.xlabel('epoch')
    plt.legend(loc='center left') # the plot evolves to the right
    plt.show();

In [11]:
import plotly.graph_objects as go
def plot(x,y,z,data = None):
    fig = go.Figure()
    fig.add_trace(go.Surface(x = x, y = y,z=z))
    if data is not None: 
        fig.add_trace(go.Scatter3d(x=data[:,0], y=data[:,1], z=data[:,2],
                                   mode='markers'))

    fig.update_layout(title='Surface Plot', autosize=True,
                  width=800, height=800, font=dict(
                  family="Courier New, monospace",
                  size=18),
                  margin=dict(l=65, r=50, b=65, t=90))
    #camera = dict(
    #up=dict(x=2, y=-0.5, z=1),
    #center=dict(x=0, y=0, z=0),
    #eye=dict(x=1.25, y=1.25, z=1.25)
    #)
    #fig.update_layout(scene_camera=camera)


    fig.show()

In [25]:
def make_plot(data):
    x1,x2 = np.linspace(-100,100,100),np.linspace(-100,100,100)
    x_pred = np.transpose([np.tile(x1, len(x2)), np.repeat(x2, len(x1))])

    x1,x2 = np.meshgrid(x1,x2)
    z = np.zeros((10000))
    c = np.zeros((10000))
    for i in range(10000): 
        z[i] = schwefel(x_pred[i], 1, 1)
        c[i] = g(x_pred[i],1,1)
    plot(x1,x2,z.reshape(100,100).T, data = data)
    plot(x1,x2,c.reshape(100,100).T)

In [26]:
import numpy as np
from hgdl.hgdl import HGDL as hgdl
from hgdl.support_functions import *
from hgdl.constraints import NonLinearConstraint
import time
import dask.distributed as distributed
import tracemalloc

def g(x, arr, barr):
    return (np.linalg.norm(x)**2/10.0) - 400.0

def g_grad(x, arr, barr):
    return (2./10.) * x

def g_hess(x, arr, barr):
    return np.identity(len(x)) * (2./10.)


con = NonLinearConstraint(g,g_grad,g_hess)



def test_schwefel():
    arr  = 5
    brr  = 6
    bounds = np.array([[-500,500],[-500,500]])
    a = hgdl(schwefel, schwefel_gradient, bounds,
            hess = schwefel_hessian,
            #global_optimizer = "random",
            global_optimizer = "genetic",
            #global_optimizer = "gauss",
            local_optimizer = "L-BFGS-B",
            number_of_optima = 30000,
            args = (arr,brr), radius = None, num_epochs = 100, constraints = (con,))

    x0 = np.random.uniform(low = np.ones((3)) * -500., high = np.ones((3)) * 500.,size = (20,3))
    print("starting positions: ")
    print(x0)
    print("--------------------")
    a.optimize(x0 = x0)



    #print(a.optima_list)
    print("main thread submitted HGDL and will now sleep for 2 seconds")
    time.sleep(2)
    print("main thread asks for 10 best solutions:")
    print(a.get_latest())
    #a.cancel_tasks()
    print("main sleeps for another 2 seconds")
    time.sleep(2)
    print("main thread kills optimization")
    res = a.kill_client()
    print("hgdl was killed but I am waiting 2s")
    print("")
    print("")
    print("")
    print("")
    print(res["x"])
    fe = np.ones(3)
    for i in range(3): fe[i] = schwefel(res["x"][i,0:2],1,1)
    print(fe)
    make_plot(np.column_stack([res["x"][:,0:2],fe]))

if __name__ == '__main__':
    test_schwefel()

starting positions: 
[[ 364.36758406  -82.72232335 -153.95931961]
 [-341.23603566  466.65286093 -469.85885206]
 [-448.58603703  -21.98273511   12.42962518]
 [  10.5840969   306.47873373  -44.75879818]
 [ 251.05492448 -227.4166156   486.25561822]
 [-497.01595676   92.90059628  320.86102696]
 [ 190.78770266  106.88832239  410.38664536]
 [ 452.47411047 -202.50714126  -11.97758068]
 [ 338.72750392  464.18862914   74.34320029]
 [ 203.36682181 -353.7921372   263.55001474]
 [ -16.14889726  353.16331115 -453.07679744]
 [ 264.51990567  285.85638828   33.86179361]
 [  -0.9674593  -119.64661034 -252.34398994]
 [  -0.64053922  167.4828681  -273.14698712]
 [ 323.96360584 -129.47427316  376.14847144]
 [  97.69001506  484.74712832    2.75790717]
 [-142.94879278 -440.49767571 -455.63257724]
 [  76.29364128  363.83413826  474.03442206]
 [-112.53752887  361.51792941 -196.57306017]
 [-366.7297769   168.59455778 -163.97538641]]
--------------------
main thread submitted HGDL and will now sleep for 2 secon

/home/marcus/VirtualEnvironments/fvgp_env/lib/python3.8/site-packages/scipy/optimize/_minimize.py:541: RuntimeWarning: Method L-BFGS-B cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,
/home/marcus/VirtualEnvironments/fvgp_env/lib/python3.8/site-packages/scipy/optimize/_minimize.py:541: RuntimeWarning: Method L-BFGS-B cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,
/home/marcus/VirtualEnvironments/fvgp_env/lib/python3.8/site-packages/scipy/optimize/_minimize.py:541: RuntimeWarning: Method L-BFGS-B cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


hgdl was killed but I am waiting 2s




[[-341.23603566  466.65286093 -469.85885206]
 [ 364.36758406  -82.72232335 -153.95931961]
 [-448.58603703  -21.98273511   12.42962518]]
[ 535.3678124   778.53257646 1141.29585111]
